# Code for generating torus knots, framings, etc

The code below is an elementary demonstration of parametric surfaces using vispy.  You are allowed to rotate the surface, and to zoom only. It generates images of what are called (p,q)-torus knots, and colours them according to the homological framing. 

**Block 1** sets up the basic information of the surface.  It is coded in sympy, and defined a surface around a (p,q) torus knot.  Everything is high-level and algebraic in this block. 

**Block 2** is set up to use four different methods to convert the sympy expression from block 1 into a function that can be called. We try these methods as the default sympy "evalf" command turns out to be *extremely* slow.  So we try lambdify, ufuncify and theano to determine the relative speed of our options.  For this task, ufuncify turns out to be the most efficient.  

**Block 3** sets up the raw data vispy needs to render the scene, using the callable function we chose in Block 2.

**Block 4** is where we communicate the data to vispy, and render. 

**TODO**: let's modify this using the vispy "canvas" to add an interactive element to the display. Future demonstrations will include:

    1) animation of the knot thickness
    2) animation of cocircular pentuples. 

In [1]:
import sympy as sp

## symbols we need to describe p,q-torus knots
## t time parameter. p,q indexes the torus knot
## r minor radius, R major radius
spt, spp, spq, spr, spR = sp.symbols("t p q r R", real=True)

c = sp.Matrix([(spR+spr*sp.cos(2*sp.pi*spq*spt))*sp.cos(2*sp.pi*spp*spt), 
     (spR+spr*sp.cos(2*sp.pi*spq*spt))*sp.sin(2*sp.pi*spp*spt), 
      spr*sp.sin(2*sp.pi*spq*spt)])
dc = sp.Matrix([sp.diff(x,spt) for x in c]) # derivative
ldc = sp.sqrt(sum( [ x**2 for x in dc ] )).simplify() # speed
udc = dc/ldc

## 2nd order
kc = sp.Matrix([sp.diff(x,spt) for x in udc]) # curvature vector
ks = sp.sqrt(sum( [ x**2 for x in kc])) # curvature scalar
ukc = kc/ks # unit curvature vector

## bi-normal
bnc = udc.cross(ukc) # cross of unit tangent and unit curvature.

## the parametrization of the boundary of the width w tubular neighbourhood

spw, spu = sp.symbols("w, u", real=True) ## width of torus knot, and meridional parameter
tSurf = c + spw*sp.cos(2*sp.pi*spu)*ukc + spw*sp.sin(2*sp.pi*spu)*bnc

In [2]:
## Let's have a visualization routine that takes as input a curve and a framing of the curve.  
##  We will then plot things like a tubular neighbourhood of the curve, together with some
##  decoration on the boundary. 

import numpy as np
import itertools as it

## (a) lambdify with numpy.  This returns a 3-element list.
knotSnp = sp.lambdify((spt, spp, spq, spr, spR, spw, spu), tSurf, "numpy" )

## (b) ufuncify
from sympy.utilities.autowrap import ufuncify
knotSuf = [ufuncify([spt, spp, spq, spr, spR, spw, spu], tSurf[i]) for i in range(3)]

## (c) theano
from sympy.printing.theanocode import theano_function
knotSth = theano_function([spt,spp,spq,spr,spR,spw,spu], [tSurf],
                          dims={spt:0, spp:0, spq:0, spr:0, spR:0, spw:0, spu:0})

## Let's have a visualization routine that takes as input a curve and a framing of the curve.  
##  We will then plot things like a tubular neighbourhood of the curve, together with some
##  decoration on the boundary. 

import numpy as np
import itertools as it

## (a) lambdify with numpy.  This returns a 3-element list.
knotSnp = sp.lambdify((spt, spp, spq, spr, spR, spw, spu), tSurf, "numpy" )

## (b) ufuncify
from sympy.utilities.autowrap import ufuncify
knotSuf = [ufuncify([spt, spp, spq, spr, spR, spw, spu], tSurf[i]) for i in range(3)]

## (c) theano
from sympy.printing.theanocode import theano_function
knotSth = theano_function([spt,spp,spq,spr,spR,spw,spu], [tSurf],
                          dims={spt:0, spp:0, spq:0, spr:0, spR:0, spw:0, spu:0})

kp = 5 ## these are the "p" and
kq = 3 ## "q" of our (p,q) torus knot.
tR = 1.6 # major torus radius
tr = 0.6 # minor torus radius. 
kt = (np.pi*tr) / (4*kp) # knot radial thickness 2*pi*tr is circumf, and kp strands pass through so this
## should be around 2*pi*tr  would be 2*kp*kt for the knot to fill the surface, i.e kt = pi*tr / 4*kp
## make bigger or smaller depending on how much empty space one wants to see.

seg = kp*300 ## segments along length of pq torus knot. kp*120 gives a fairly smooth image.
segm = 40 ## meridional segmentation of pq torus knot. 60 is fairly smooth. 

def surf1(i,j): ## sympy raw
    return np.array(tSurf.evalf(subs={spt:float(i)/seg, spu:float(j)/segm, 
                                       spp:kp, spq:kq, spr:tr, spR:tR, spw:kt}) )
def surf2(i,j): ## lambdify
    return np.array(knotSnp(float(i)/seg, kp, kq, tr, tR, kt, float(j)/segm)).ravel()
def surf3(i,j): ## ufuncify
    return np.array([knotSuf[k](float(i)/seg, kp, kq, tr, tR, kt, float(j)/segm) for k in range(3)])
def surf4(i,j): ## theano
    return knotSth(float(i)/seg, kp, kq, tr, tR, kt, float(j)/segm).ravel()

Surf = [surf1, surf2, surf3, surf4]
SurfLabel = ["sympy.evalf", "sympy.lambdify", "ufuncify", "theano"]

Using gpu device 0: Quadro K2000M (CNMeM is disabled)


In [3]:
k = 2 # determines which method we use to cast sympy expressions to a callable function.

import time as ti
start=ti.time()
surf = Surf[k]
pqMesh = np.array(list( it.chain.from_iterable(([surf(i,j), surf(i+1,j), surf(i,j+1)], 
                                                [surf(i+1,j), surf(i+1,j+1), surf(i,j+1)]) for i,j in it.product(range(seg), range(segm)))))
end=ti.time()
print(SurfLabel[k]+" mesh generation: "+str(end-start)+" seconds.", flush=True)

colA = np.array([0.0, 0.0, 1.0, 1.0])
colB = np.array([1.0, 1.0, 0.0, 1.0])

#print(SurfLabel[useFunc] + str(" mesh ")+str(pqMesh))

def iCol(i,j): ## interpolates between colA and colB
    alph = np.pi*float(j)/segm
    bet = np.pi*float(i)/seg
    gam = 1.0*alph - kp*kq*bet ## coprime combination of alpha and beta
    return (np.sin(gam)**2)*colA + (np.cos(gam)**2)*colB
    
## coloration of mesh
pqColors = np.array(list(it.chain.from_iterable(([iCol(i,j),iCol(i+1,j),iCol(i,j+1)], 
                                              [iCol(i+1,j),iCol(i+1,j+1),iCol(i,j+1)]) for i,j in it.product(range(seg), range(segm)) )))

ufuncify mesh generation: 12.16636872291565 seconds.


In [4]:
## Let's now generate the vispy data
from vispy import geometry, app, scene
import vispy, sys
from vispy.scene import cameras

canvas = scene.SceneCanvas(keys='interactive', size=(800, 600), show=True)
view = canvas.central_widget.add_view()
view.parent = canvas.scene
view.camera = scene.TurntableCamera(parent=view.scene, fov=45)  
view.bgcolor = '#606060'
## shading options are empty (pass nothing), shading='smooth' and shading='flat'
pqtorus = scene.visuals.Mesh(pqMesh,face_colors=pqColors, shading='smooth')

axis = scene.visuals.XYZAxis(parent=view.scene)

## and showtime! 
view.add(pqtorus)
canvas.app.run()

INFO: Could not import backend "PyQt4":
cannot import name 'QtOpenGL'
INFO:vispy:Could not import backend "PyQt4":
cannot import name 'QtOpenGL'
INFO: Could not import backend "PyQt5":
cannot import name 'QtOpenGL'
INFO:vispy:Could not import backend "PyQt5":
cannot import name 'QtOpenGL'
/usr/local/lib/python3.4/dist-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)


0